In [1]:
import pandas as pd
import numpy as np
import setup_jwlab
from jwlab.constants import cleaned_data_filepath
from jwlab.cluster_analysis import prep_cluster_analysis
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from scipy import stats
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score


In [2]:
length_per_window = 10
num_sliding_windows = int(1200/ length_per_window)

In [3]:
participants = ["904", "905", "906", "908", "909", "912","913", "914", "916", "917", "919",\
                "920", "921", "923", "924", "927", "928", "929", "930", "932"]

In [ ]:
participants = ["107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124"]


In [ ]:
participants = ["107", "109", "111", "112", "115", "116", "117", "119", "121", "122", "120", "124",\
               "904", "905", "906", "908", "909", "912","913", "914", "916", "917", "919",\
                "920", "921", "923", "924", "927", "928", "929", "930", "932"]


In [ ]:
for i in participants:
    ps =[i]
    X, y, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, ps, downsample_num=1000, averaging="no_averaging", length_per_window=length_per_window)
    y[0]
    np.amax(y[0])
    unique, counts = np.unique(y[0], return_counts=True)
    
    print(i)
    print(dict(zip(unique, counts)))

## Train raw test avg

In [ ]:
X, y, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1200, averaging="no_averaging", length_per_window=length_per_window)
Xt, yt, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1200, averaging="average_trials", length_per_window=length_per_window)
Xa, ya, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1200, averaging="average_trials_and_participants", length_per_window=length_per_window)

model = LinearSVC(C=1e-9, max_iter=5000)
#model = SVC(gamma=.001, kernel = 'rbf', C = 1e-6)

# from sklearn import preprocessing


results = {}



for j in range(num_sliding_windows):
    
        
        model.fit(X[j], y[j])
        
        # validation, predict raw
#         y_pred = model.predict(X[j])
#         testScore = accuracy_score(y[j],y_pred)
        
        #predict averaged across trials
#         y_pred = model.predict(Xt[j])
#         testScore = accuracy_score(yt[j],y_pred)
        
        # predict averaged across trials and ps
        y_pred = model.predict(Xa[j])
        testScore = accuracy_score(ya[j],y_pred)




    

        if j in results.keys(): 
            results[j].append(testScore)
        else:
            results[j]=[]
            results[j].append(testScore)
    


In [ ]:
X[j]

In [ ]:
y[0]
np.amax(y[0])
unique, counts = np.unique(y[0], return_counts=True)
dict(zip(unique, counts))

In [ ]:
len(X)

In [ ]:
num_sliding_windows

In [ ]:
# # Cross validation

#X, y, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="no_averaging", length_per_window=length_per_window)

# for j in range(num_sliding_windows):
#     #model = SVC(gamma=.001, kernel = 'rbf', C=1000)
#     model = LinearSVC(C=1e-9, max_iter=5000)
#     cv_results = cross_validate(model, X[j], y[j], cv=num_folds)
#     if j in results.keys(): 
#         results[j] += cv_results['test_score'].tolist()
#     else:
#         results[j] = cv_results['test_score'].tolist()
    

# for i in range(num_sliding_windows):
#     assert len(results[i]) == num_iterations * num_folds

In [ ]:
# Randomized order of cross val, for raw data matrix

X, y, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1200, averaging="no_averaging", length_per_window=length_per_window)


num_iterations = 5
num_folds = 5

results = {}
for i in range(num_iterations):
    for j in range(num_sliding_windows):
        Xfirst = X[j]
        yfirst = y[j]
        Xfirst['label'] = yfirst
        Xfirst = Xfirst.sample(frac=1).reset_index(drop=True)
        ys = Xfirst['label']
        Xs = Xfirst.drop(columns=['label'])
        
        #model = SVC(gamma=.001, kernel = 'rbf', C=1)
        model = LinearSVC(C=1e-9, max_iter=5000)
        cv_results = cross_validate(model, Xs, ys, cv=num_folds)
        if j in results.keys(): 
            results[j] += cv_results['test_score'].tolist()
        else:
            results[j] = cv_results['test_score'].tolist()
    print(i)

    
for i in range(num_sliding_windows):
    assert len(results[i]) == num_iterations * num_folds


In [ ]:
X[0].shape
X.shape

In [4]:
# Cross validation with RepeatedKFold for averaged matrices

#Xwin, ywin, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials", length_per_window=length_per_window)
Xwin, ywin, good_trial_count = prep_cluster_analysis(cleaned_data_filepath, participants, downsample_num=1000, averaging="average_trials_and_participants", length_per_window=length_per_window)


num_iterations = 3
num_folds = 5


import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score



results = {}
for j in range(num_sliding_windows):
    X = Xwin[j]
    y = ywin[j]

    #model = SVC(gamma=.001, kernel = 'rbf', C=1e-4)
    model = LinearSVC(C=1e-9, max_iter=5000)
    rkf = RepeatedKFold(n_splits=num_folds, n_repeats=num_iterations, random_state=2652124)
    for train_index, test_index in rkf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        testScore = accuracy_score(y_test,y_pred)

        if j in results.keys(): 
            results[j].append(testScore)
        else:
            results[j]=[]
            results[j].append(testScore)
    

for i in range(num_sliding_windows):
    assert len(results[i]) == num_iterations * num_folds

loaded


In [5]:
scoreMean = []
stdev = []

for i in range(num_sliding_windows):
    scoreMean.append(np.mean(results[i]))
    stdev.append(np.std(results[i]))



In [6]:
np.mean(scoreMean)

0.36962962962962964

In [7]:
scoreMean

[0.5666666666666667,
 0.4666666666666667,
 0.4888888888888889,
 0.5888888888888889,
 0.5222222222222223,
 0.41111111111111115,
 0.4,
 0.27222222222222225,
 0.3777777777777777,
 0.49444444444444435,
 0.5722222222222223,
 0.5888888888888889,
 0.5833333333333333,
 0.6277777777777777,
 0.5888888888888888,
 0.3722222222222222,
 0.4166666666666667,
 0.3888888888888889,
 0.3111111111111111,
 0.26666666666666666,
 0.27222222222222225,
 0.31111111111111106,
 0.3611111111111111,
 0.32222222222222224,
 0.2722222222222222,
 0.2944444444444444,
 0.29444444444444445,
 0.27777777777777773,
 0.31666666666666665,
 0.31666666666666665,
 0.2777777777777778,
 0.31666666666666665,
 0.3111111111111111,
 0.27222222222222225,
 0.2722222222222222,
 0.29444444444444445,
 0.3277777777777778,
 0.3722222222222222,
 0.4111111111111111,
 0.42777777777777776,
 0.3333333333333333,
 0.3333333333333333,
 0.31666666666666665,
 0.29444444444444445,
 0.29444444444444445,
 0.29444444444444445,
 0.3388888888888889,
 0.3,
 0.

In [8]:
stdev

[0.21773242158072695,
 0.17159383568311665,
 0.24507494444569752,
 0.2182986967154278,
 0.21184084405486245,
 0.26258449551325497,
 0.24190601174530268,
 0.22457092146008598,
 0.21271321910085164,
 0.29260431293919315,
 0.2561587085199667,
 0.29731307022799225,
 0.305807976544953,
 0.23346557099389662,
 0.24241582476968254,
 0.1992269008560123,
 0.2527625148017183,
 0.15713484026367724,
 0.18121673811444547,
 0.1699673171197595,
 0.1442306109527463,
 0.09844469525927414,
 0.2446968393949471,
 0.24507494444569752,
 0.2051798368068821,
 0.227438772116208,
 0.24507494444569755,
 0.25579698740491863,
 0.23213980461973532,
 0.2621139136018989,
 0.25579698740491863,
 0.26211391360189884,
 0.2499382639822665,
 0.22457092146008598,
 0.22457092146008595,
 0.2127132191008516,
 0.2695446549046267,
 0.2702308112670069,
 0.22865684030285197,
 0.2431785403137699,
 0.25276251480171835,
 0.25276251480171835,
 0.23213980461973532,
 0.2127132191008516,
 0.2127132191008516,
 0.2127132191008516,
 0.248079

In [ ]:
# T-test
accuracy_by_guessing = [0.5] * (num_iterations * num_folds)
pvalues = []
for i in range(num_sliding_windows):
    istat = stats.ttest_1samp(results[i], .5)
    pvalues += [istat.pvalue] if istat.statistic > 0 else [1]

In [ ]:
# Finding contiguous time cluster
valid_window = [i for i,v in enumerate(pvalues) if v <= 0.025]
print(valid_window)